In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
#from datetime import datetime

In [2]:
df = pd.read_csv('../datas/train_passengers_copy.csv')
#Variables for training
cols = list(df)[1:3]
df_for_training = df[cols].astype(float)

In [3]:
#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
scaler = MinMaxScaler()
scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [4]:
#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 5. We will make timesteps = 14 (past days data used for training). 

#Empty lists to be populated using formatted training data
trainX = []
trainY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 3  # Number of past days we want to use to predict the future.

In [5]:
#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (141, 3, 2).
trainY shape == (141, 1).


In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(3, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 3)              72        
                                                                 
 dense (Dense)               (None, 3, 1)              4         
                                                                 
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(trainX, trainY, epochs=100, batch_size=16, validation_split=0.1, verbose=1)

Epoch 1/100
8/8 [==============================] - 4s 139ms/step - loss: 0.1731 - val_loss: 0.5358
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 0.1582 - val_loss: 0.5032
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 0.1442 - val_loss: 0.4719
Epoch 4/100
8/8 [==============================] - 0s 10ms/step - loss: 0.1312 - val_loss: 0.4419
Epoch 5/100
8/8 [==============================] - 0s 11ms/step - loss: 0.1192 - val_loss: 0.4134
Epoch 6/100
8/8 [==============================] - 0s 7ms/step - loss: 0.1082 - val_loss: 0.3862
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: 0.0975 - val_loss: 0.3613
Epoch 8/100
8/8 [==============================] - 0s 8ms/step - loss: 0.0889 - val_loss: 0.3367
Epoch 9/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0802 - val_loss: 0.3145
Epoch 10/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0728 - val_loss: 0.2936
Epoch 11/100
8/8 [=======

In [8]:
# model.save('../saved_param/model3')


In [9]:
load_model = tf.keras.models.load_model("../saved_param/model3")


In [10]:
load_model.predict(np.array([trainX[1]]))

1/1 [==============================] - 1s 1s/step


array([[[0.25702322],
        [0.29458588],
        [0.30806884]]], dtype=float32)

In [11]:
trainX[0]

array([[0.01544402, 0.8       ],
       [0.02702703, 0.8       ],
       [0.05405405, 0.6       ]])

In [12]:
load_model.layers[1].trainable_weights

[<tf.Variable 'dense_3/kernel:0' shape=(3, 1) dtype=float32, numpy=
 array([[-0.42233184],
        [ 0.3689297 ],
        [-0.3131978 ]], dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(1,) dtype=float32, numpy=array([0.16121146], dtype=float32)>]

In [13]:
W = load_model.layers[0].get_weights()[0]
U = load_model.layers[0].get_weights()[1]
b = load_model.layers[0].get_weights()[2]

W_i = W[:, :3]
W_f = W[:, 3: 3 * 2]
W_c = W[:, 3 * 2: 3 * 3]
W_o = W[:, 3 * 3:]

U_i = U[:, :3]
U_f = U[:, 3: 3 * 2]
U_c = U[:, 3 * 2: 3 * 3]
U_o = U[:, 3 * 3:]

b_i = b[:3]
b_f = b[3: 3 * 2]
b_c = b[3 * 2: 3 * 3]
b_o = b[3 * 3:]

In [14]:
print(W_i)
print(U_i)
print(b_i)


[[0.35566998 1.4086083  0.7215156 ]
 [0.07884085 0.36922053 0.37740988]]
[[-0.11504292 -0.11362755 -0.3031734 ]
 [-0.9906854  -0.23550205 -0.26340267]
 [ 0.41692337  0.14517373  0.17972483]]
[-0.16188446  0.01601927  0.21576458]


In [15]:
print(W_f)
print(U_f)
print(b_f)


[[-0.2295345   1.1335361   0.6621852 ]
 [ 0.25860325  0.1935192  -0.30316782]]
[[ 0.47441137  0.14656599  0.08143887]
 [-0.45709613 -0.31698605 -0.01739863]
 [-0.11745029  0.11935769  0.39047447]]
[0.87010735 0.74171436 0.87208897]


In [16]:
print(W_c)
print(U_c)
print(b_c)


[[ 0.2736181   0.3080478  -0.20788296]
 [ 0.42466313  0.403848   -0.5905587 ]]
[[-0.50114334  0.09257057 -0.19414988]
 [-0.14574955  0.11357803  0.19666846]
 [-0.42942908  0.6167454  -0.29570833]]
[-0.08857886  0.19663301 -0.25165126]


In [17]:
print(W_o)
print(U_o)
print(b_o)


[[-0.74122727  0.46246535  0.4048134 ]
 [ 0.33073547 -0.1942196   0.03327436]]
[[-0.23198946  0.16087262  0.59650433]
 [ 0.1249354   0.44169274  0.20607027]
 [ 0.48772782 -0.2447438   0.36812192]]
[-0.17079982  0.29212683  0.26432905]


In [18]:
# test_model = tf.keras.Sequential()
# test_model.add(tf.keras.layers.LSTM(3, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))

In [19]:
# test_model.save('../saved_param/test_model1')

In [20]:
test_model = tf.keras.models.load_model("../saved_param/test_model1/")

In [21]:
trainX[1]

array([[0.02702703, 0.8       ],
       [0.05405405, 0.6       ],
       [0.04826255, 0.6       ]])

In [22]:
test_model.predict(np.array([trainX[1]]))

1/1 [==============================] - 1s 1s/step


array([[[-0.00752289,  0.04412096, -0.0144942 ],
        [-0.01602702,  0.06958452, -0.02986364],
        [-0.01964575,  0.08610261, -0.04356184]]], dtype=float32)

In [23]:
test_model.layers[0].trainable_weights

[<tf.Variable 'lstm_6/lstm_cell_7/kernel:0' shape=(2, 12) dtype=float32, numpy=
 array([[ 0.28213406,  0.40180504,  0.10323656,  0.49266553,  0.59737396,
         -0.27416527, -0.6140406 , -0.22036755, -0.36234343,  0.11618108,
         -0.40446782, -0.05731392],
        [ 0.09265673, -0.55035496, -0.0250091 ,  0.10383672, -0.03438932,
          0.44054282, -0.02767795,  0.35591948, -0.076204  , -0.6510474 ,
         -0.4240626 , -0.43181473]], dtype=float32)>,
 <tf.Variable 'lstm_6/lstm_cell_7/recurrent_kernel:0' shape=(3, 12) dtype=float32, numpy=
 array([[-0.11440527, -0.09116384,  0.10766367,  0.41458526,  0.13482785,
          0.02425341, -0.14716567,  0.09884649, -0.07846723, -0.2423919 ,
          0.7091871 ,  0.42080373],
        [-0.09434418,  0.6440063 , -0.16892694,  0.09986253, -0.29395232,
         -0.35163632, -0.18524946,  0.01135916, -0.2985677 ,  0.35922354,
          0.23562133, -0.1401508 ],
        [ 0.12497412,  0.13285607,  0.29592115, -0.17216626, -0.05532025,
  

In [24]:
# wrapped_test_model = tf.keras.Sequential()
# wrapped_test_model.add(test_model)
# wrapped_test_model.add(tf.keras.layers.Dense(1))

In [25]:
wrapped_test_model.layers[1].get_weights()

NameError: name 'wrapped_test_model' is not defined

In [ ]:
wrapped_test_model.predict(np.array([trainX[1]]))

1/1 [==============================] - 0s 39ms/step


array([[[-0.03293667],
        [-0.04143049],
        [-0.04421219]]], dtype=float32)